In [1]:
from dotenv import load_dotenv
import os
import requests
from bs4 import BeautifulSoup
import cloudscraper
import re

load_dotenv()

True

In [122]:
class Source:

    @staticmethod
    def get_soup_object(flag_mock: bool) -> BeautifulSoup:
        '''
        Gets data from os.getenv() and returns beautiful Soup object.

        If flag_mock = True, it returns the contents of a local file to prevent making requests during tests.
        '''
        soup_obj = None
        response = str()

        if flag_mock:
            with open('test.html', 'r') as f:
                response =  f.read()
        else:
            source = os.getenv('SOURCE_1')
            scraper = cloudscraper.create_scraper()
            response = scraper.get(source).text

        soup = BeautifulSoup(response, 'html.parser')
        return soup
    
    @staticmethod
    def get_raw_question_data(soup: BeautifulSoup) -> dict:
        '''
        Scrapes the soup object to get text for p elements which contain question and answer.
        '''

        ptags = soup.find_all('p')
        regex_question = r'^\d+\..*'
        regex_answer = r'^Question.*'
        relevant_info = {'Questions': [], 'Answers': []}
        for ptag in ptags:
            re_match_questions = re.match(regex_question, ptag.text)
            re_match_answers = re.match(regex_answer, ptag.text)
            if re_match_questions:
                relevant_info['Questions'].append(re_match_questions.group(0))
                continue
            if re_match_answers:
                relevant_info['Answers'].append(re_match_answers.group(0))

        relevant_info['Answers'] = relevant_info['Answers'][0].split('Question ')[1:]

        return relevant_info
    
    @staticmethod
    def extract_question_str_choices(question_text: str) -> dict:
        '''
        Create a dictionary with question text and question choices
        '''
        match = re.match(r'^(\d+)\.\s*(.*?)\s*([^。]+)$', question_text)
        if match:
            question_number = int(match.group(1).strip())
            question_str = match.group(2).strip()  
            question_choices = match.group(3).strip()    
            
            question_choices = question_choices.split(' ')

            question_choices = {str(index+1): choice for index, choice in enumerate(question_choices)}
            question_choices = str(question_choices)

            return {'question_number': question_number, 'question_str': question_str, 'question_choices': question_choices}
            
        return dict()
    
    @staticmethod
    def extract_question_answer(answer_text: str):
        '''
        Create a tuple containing the answer of question
        '''
        pattern = r'^(\d+):\s(\d+).*$'
        result = re.match(pattern, answer_text)
        if result:
            return (result.group(1), result.group(2))

In [123]:
soup = Source.get_soup_object(flag_mock=True)
raw_question_data = Source.get_raw_question_data(soup)
raw_question_data

{'Questions': ['1. ケネディ殺害の容疑者は _______ に謎を残したままマフィアに撃たれて死亡した。 動機 本音 動力 下心',
  '2. いつ見つけても _______ の早いがんでは予後が悪く、遅いがんは予後がいい。早くても遅くても意味はないのです。 先進 増進 進出 進行',
  '3. インターネット広告 _______ と広告の効果の関係について考えてみよう。 値 費 料 額',
  '4. この数年間で千葉や隣接県では女児連れ去り事件が多発していた。まだ _______ 解決の案件もあるが、未遂に終わり、容疑者が逮捕されたケースもある。 非 双 未 無',
  '5. 辞書を引いたら、最初に _______ されている語義だけでなく、すべての語義をざっと確認する習慣を付けましょう。 記載 援用 参照 出典',
  '6. 専門学校で、デザインという仕事の難しさに _______ することになりました。 対決 直面 抵抗 挑戦',
  '7. この機械を _______ にはかなりの技術が必要だ。 使いこなす 使いおわる 使いはたす 使いすてる',
  '8. 最近の青少年はしっかりしているようだが、精神的に _______ 面がある。 しぶい だるい ゆるい もろい',
  '9. 何回会議をやっても結論が出ないので、_______ いやになった。 つくづく わざわざ ぞくぞくと くれぐれも',
  '10. 不況の影響で、この地域の中小企業は _______ 倒産した。 いまさら ひたすら のきなみ ひいては',
  '11. 年をとると、だんだん新しい考え方が _______ にくくなる。 うけあい うけいれ うけとり うけもち',
  '12. このアンケートに協力するかしないかは自由で、________ はしないということにしたい。 圧迫 強制 催促 一致'],
 'Answers': ['1: 1 (ケネディ殺害の容疑者は動機に謎を残したままマフィアに撃たれて死亡した。)',
  '2: 4 (いつ見つけても進行の早いがんでは予後が悪く、遅いがんは予後がいい。早くても遅くても意味はないのです。)',
  '3: 2 (インターネット広告費と広告の効果の関係について考えてみよう。)',
  '4: 3 (この数年間で千葉や

In [124]:
questions, answers = raw_question_data['Questions'], raw_question_data['Answers']

myQuestions = list()

for question, answer in zip(questions, answers):
    question_data = Source.extract_question_str_choices(question)
    question_answer = Source.extract_question_answer(answer)
    
    if question_data['question_number'] == int(question_answer[0]):
        question_data['question_answer'] = question_answer[1]

        del question_data['question_number']
        myQuestions.append(question_data)

In [127]:
myQuestions

[{'question_str': 'ケネディ殺害の容疑者は _______ に謎を残したままマフィアに撃たれて死亡した。',
  'question_choices': "{'1': '動機', '2': '本音', '3': '動力', '4': '下心'}",
  'question_answer': '1'},
 {'question_str': 'いつ見つけても _______ の早いがんでは予後が悪く、遅いがんは予後がいい。早くても遅くても意味はないのです。',
  'question_choices': "{'1': '先進', '2': '増進', '3': '進出', '4': '進行'}",
  'question_answer': '4'},
 {'question_str': 'インターネット広告 _______ と広告の効果の関係について考えてみよう。',
  'question_choices': "{'1': '値', '2': '費', '3': '料', '4': '額'}",
  'question_answer': '2'},
 {'question_str': 'この数年間で千葉や隣接県では女児連れ去り事件が多発していた。まだ _______ 解決の案件もあるが、未遂に終わり、容疑者が逮捕されたケースもある。',
  'question_choices': "{'1': '非', '2': '双', '3': '未', '4': '無'}",
  'question_answer': '3'},
 {'question_str': '辞書を引いたら、最初に _______ されている語義だけでなく、すべての語義をざっと確認する習慣を付けましょう。',
  'question_choices': "{'1': '記載', '2': '援用', '3': '参照', '4': '出典'}",
  'question_answer': '1'},
 {'question_str': '専門学校で、デザインという仕事の難しさに _______ することになりました。',
  'question_choices': "{'1': '対決', '2': '直面', '3': '抵抗', '4': '挑戦'}",
  'question_a